In [ ]:
# путь до данных на компьютере
path = '/content/drive/MyDrive/files_and_description/train_data'

In [ ]:
# для работающих в Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

## Чтение в 338 признаков

In [ ]:
dataset_paths = sorted([os.path.join(path, filename) for filename in os.listdir(path)
                            if filename.startswith('train')])
print(dataset_paths)

['/content/drive/MyDrive/files_and_description/train_data/train_data_0.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_1.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_10.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_11.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_2.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_3.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_4.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_5.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_6.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_7.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_8.pq', '/content/drive/MyDrive/files_and_description/train_data/train_data_9.pq']


Так как признаки бинаризованы (по сути, категориальные несравнимые признаки) и имеют малое количество уникальных значений, используем OHE-кодирование. При этом: удалим 4 признака про сроки сбора информации и к признакам, имеющих 2 уникальных значения, ohe-кодирование применять не будем (это признаки - флаги).

In [ ]:
# Сделаем эти списки постоянными, чтобы не зависеть от вариативности данных в разных датасетах
bi_cols = ['is_zero_loans5', 'is_zero_loans530', 'is_zero_loans3060', 'is_zero_loans6090', 'is_zero_loans90', 'is_zero_util', 'is_zero_over2limit', 'is_zero_maxover2limit', 'pclose_flag', 'fclose_flag']
cat_cols = ['pre_pterm', 'pre_fterm', 'pre_loans_credit_limit', 'pre_loans_next_pay_summ', 'pre_loans_outstanding', 'pre_loans_max_overdue_sum', 'pre_loans_credit_cost_rate', 'pre_loans5', 'pre_loans530', 'pre_loans3060', 'pre_loans6090', 'pre_loans90', 'pre_util', 'pre_over2limit', 'pre_maxover2limit', 'enc_paym_0', 'enc_paym_1', 'enc_paym_2', 'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7', 'enc_paym_8', 'enc_paym_9', 'enc_paym_10', 'enc_paym_11', 'enc_paym_12', 'enc_paym_13', 'enc_paym_14', 'enc_paym_15', 'enc_paym_16', 'enc_paym_17', 'enc_paym_18', 'enc_paym_19', 'enc_paym_20', 'enc_paym_21', 'enc_paym_22', 'enc_paym_23', 'enc_paym_24', 'enc_loans_account_holder_type', 'enc_loans_credit_status', 'enc_loans_credit_type', 'enc_loans_account_cur']

In [ ]:
def bi_col_trans(data, cols, id='id', rn='rn'):
    trans_data = data.groupby(id)[rn].max().reset_index(drop=False)
    trans_data = pd.merge(trans_data, data.groupby(id)[[*cols]].sum().reset_index(drop=False), left_on=id, right_on=id)
    return trans_data

In [ ]:
def ohe_trans(data, col):
    ohe = OneHotEncoder(sparse=False)
    df = pd.DataFrame(data[col], columns=[col])
    ohe.fit(df)
    ohe_categorical = ohe.transform(df)
    data[ohe.get_feature_names_out()] = ohe_categorical.astype('int8')
    data = data.drop(col, axis=1)
    data = data.groupby('id')[ohe.get_feature_names_out()].sum().reset_index(drop=False)
    return data

In [ ]:
step = 0
for file_path in dataset_paths:

    df = pd.read_parquet(file_path)
    for col in ['pre_since_opened', 'pre_since_confirmed', 'pre_till_pclose', 'pre_till_fclose', 'pre_loans_total_overdue']:
        df = df.drop(col, axis=1)
    df[bi_cols] = df[bi_cols].astype('int8')
    df[cat_cols] = df[cat_cols].astype('int8')

    trans_data = bi_col_trans(df, bi_cols)

    for col in cat_cols:
        tmp_data = ohe_trans(df[['id', col]], col)
        trans_data = pd.merge(trans_data, tmp_data, left_on='id', right_on='id')
        del tmp_data

    block_as_str = str(step)
    if len(block_as_str) == 1:
        block_as_str = '00' + block_as_str
    else:
        block_as_str = '0' + block_as_str
    trans_data.to_parquet(os.path.join(path, f'processed_file_{block_as_str}.parquet'))
    step += 1
    del df
    del trans_data

Склейка обработанных файлов

In [ ]:
prep_paths = sorted([os.path.join(path, filename) for filename in os.listdir(path)
                            if filename.startswith('processed')])
print(prep_paths)

['/content/drive/MyDrive/files_and_description/train_data/processed_file_000.parquet', '/content/drive/MyDrive/files_and_description/train_data/processed_file_001.parquet', '/content/drive/MyDrive/files_and_description/train_data/processed_file_002.parquet', '/content/drive/MyDrive/files_and_description/train_data/processed_file_003.parquet', '/content/drive/MyDrive/files_and_description/train_data/processed_file_004.parquet', '/content/drive/MyDrive/files_and_description/train_data/processed_file_005.parquet', '/content/drive/MyDrive/files_and_description/train_data/processed_file_006.parquet', '/content/drive/MyDrive/files_and_description/train_data/processed_file_007.parquet', '/content/drive/MyDrive/files_and_description/train_data/processed_file_008.parquet', '/content/drive/MyDrive/files_and_description/train_data/processed_file_009.parquet', '/content/drive/MyDrive/files_and_description/train_data/processed_file_010.parquet', '/content/drive/MyDrive/files_and_description/train_d

In [ ]:
result = pd.read_parquet(prep_paths[0])
for path in prep_paths[1:]:
    df = pd.read_parquet(path)
    result = pd.concat([result, df])
    result = result.fillna(0)
    del df


In [ ]:
result.shape

(3000000, 338)

In [ ]:
result = result.sort_values(by='id')

In [ ]:
result.to_parquet(os.path.join(path, f'fin_proc.parquet'))

In [ ]:
df = pd.read_parquet(path + '/fin_proc.parquet')

In [ ]:
df.head()

,id,rn,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,...,enc_loans_credit_type_7,pre_loans530_8,pre_loans530_9,pre_loans_max_overdue_sum_0,pre_loans3060_0,pre_loans3060_3,pre_loans3060_4,pre_loans6090_0,pre_loans5_10,pre_loans530_17
0,0,10,9,10,10,10,10,6,9,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,14,12,10,12,12,11,10,12,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3,3,2,2,2,3,1,3,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,15,15,15,15,15,15,8,14,14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
targets = pd.read_csv('/content/drive/MyDrive/files_and_description/train_target.csv')
targets.head()

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
fin_df = pd.merge(df, targets, left_on='id', right_on='id')

In [ ]:
fin_df.head()

,id,rn,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,...,pre_loans530_8,pre_loans530_9,pre_loans_max_overdue_sum_0,pre_loans3060_0,pre_loans3060_3,pre_loans3060_4,pre_loans6090_0,pre_loans5_10,pre_loans530_17,flag
0,0,10,9,10,10,10,10,6,9,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,14,12,10,12,12,11,10,12,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2,3,3,2,2,2,3,1,3,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,15,15,15,15,15,15,8,14,14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,4,1,1,1,1,1,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
fin_df.isna().sum()

id                   0
rn                   0
is_zero_loans5       0
is_zero_loans530     0
is_zero_loans3060    0
                    ..
pre_loans3060_4      0
pre_loans6090_0      0
pre_loans5_10        0
pre_loans530_17      0
flag                 0
Length: 339, dtype: int64

In [ ]:
X = fin_df.drop(['flag'], axis=1)
y = fin_df[['id', 'flag']]

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
x_train.to_parquet(os.path.join(path, f'x_train.parquet'))
x_test.to_parquet(os.path.join(path, f'x_test.parquet'))
y_train.to_parquet(os.path.join(path, f'y_train.parquet'))
y_test.to_parquet(os.path.join(path, f'y_test.parquet'))

In [ ]:
x_train.to_csv(os.path.join(path, f'x_train.csv'), index=False)
x_test.to_csv(os.path.join(path, f'x_test.csv'), index=False)
y_train.to_csv(os.path.join(path, f'y_train.csv'), index=False)
y_test.to_csv(os.path.join(path, f'y_test.csv'), index=False)

In [ ]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2100000 entries, 2870337 to 2219110
Columns: 338 entries, id to pre_loans530_17
dtypes: float64(25), int64(2), int8(311)
memory usage: 1.0 GB


## Downsampling in 338 feat

In [ ]:
df = pd.read_parquet(path + '/fin_proc.parquet')

In [ ]:
targets = pd.read_csv('/content/drive/MyDrive/files_and_description/train_target.csv')
targets.head()

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
fin_df = pd.merge(df, targets, left_on='id', right_on='id')

In [ ]:
df_min = fin_df[fin_df['flag'] == 1]
df_maj = fin_df[fin_df['flag'] == 0]

df_maj_downsample = resample(df_maj, replace=False, n_samples=len(df_min), random_state=1)

print(df_maj_downsample.shape)
print(df_min.shape)

(106442, 339)
(106442, 339)


In [ ]:
df_2 = pd.concat([df_maj_downsample, df_min], ignore_index=True).sample(frac=1.)

In [ ]:
X = df_2.drop(['flag'], axis=1)
y = df_2[['id', 'flag']]

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
x_train.to_parquet(os.path.join(path, f'ds_x_train.parquet'))
x_test.to_parquet(os.path.join(path, f'ds_x_test.parquet'))
y_train.to_parquet(os.path.join(path, f'ds_y_train.parquet'))
y_test.to_parquet(os.path.join(path, f'ds_y_test.parquet'))

## Down_upsampling in 338 feat

In [ ]:
df = pd.read_parquet(path + '/fin_proc.parquet')

In [ ]:
targets = pd.read_csv('/content/drive/MyDrive/files_and_description/train_target.csv')
targets.head()

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
fin_df = pd.merge(df, targets, left_on='id', right_on='id')

In [ ]:
df_min = fin_df[fin_df['flag'] == 1]
df_maj = fin_df[fin_df['flag'] == 0]

df_maj_downsample = resample(df_maj, replace=False, n_samples=len(df_min) * 5, random_state=1)
df_min_upsample = resample(df_min, replace=True, n_samples=len(df_maj_downsample), random_state=1)

print(df_maj_downsample.shape)
print(df_min_upsample.shape)

(532210, 339)
(532210, 339)


In [ ]:
df_2 = pd.concat([df_maj_downsample, df_min_upsample], ignore_index=True).sample(frac=1.)

In [ ]:
X = df_2.drop(['flag'], axis=1)
y = df_2[['id', 'flag']]

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
x_train.to_parquet(os.path.join(path, f'up_ds_x_train.parquet'))
x_test.to_parquet(os.path.join(path, f'up_ds_x_test.parquet'))
y_train.to_parquet(os.path.join(path, f'up_ds_y_train.parquet'))
y_test.to_parquet(os.path.join(path, f'up_ds_y_test.parquet'))

## Donsampl and SMOTE in 338 feat

In [ ]:
df = pd.read_parquet(path + '/fin_proc.parquet')

In [ ]:
targets = pd.read_csv('/content/drive/MyDrive/files_and_description/train_target.csv')
targets.head()

,id,flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [ ]:
fin_df = pd.merge(df, targets, left_on='id', right_on='id')

In [ ]:
df_min = fin_df[fin_df['flag'] == 1]
df_maj = fin_df[fin_df['flag'] == 0]

df_maj_downsample = resample(df_maj, replace=False, n_samples=len(df_min) * 3, random_state=1)

print(df_maj_downsample.shape)
print(df_min.shape)

(319326, 339)
(106442, 339)


In [ ]:
df_2 = pd.concat([df_maj_downsample, df_min], ignore_index=True).sample(frac=1.)

In [ ]:
X = df_2.drop(['flag'], axis=1)
y = df_2['flag']

osmt = SMOTE(random_state=1, k_neighbors=2)

X, y = osmt.fit_resample(X, y)

In [ ]:
y = pd.DataFrame(y, columns=['flag'] )
y.reset_index(drop=False, inplace=True)
y['id'] = y['index']
y = y.drop(['index'], axis=1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
import os

x_train.to_parquet(os.path.join(path, f'smote_x_train.parquet'))
x_test.to_parquet(os.path.join(path, f'smote_x_test.parquet'))
y_train.to_parquet(os.path.join(path, f'smote_y_train.parquet'))
y_test.to_parquet(os.path.join(path, f'smote_y_test.parquet'))

## Tecnical

In [ ]:
df = pd.read_parquet(dataset_paths[0])

In [ ]:
df.head()

In [ ]:
for col in ['pre_since_opened', 'pre_since_confirmed', 'pre_till_pclose', 'pre_till_fclose', 'pre_loans_total_overdue']:
    df = df.drop(col, axis=1)

In [ ]:
df[bi_cols] = df[bi_cols].astype('int8')
df[cat_cols] = df[cat_cols].astype('int8')

In [ ]:
df.info()

In [ ]:
def bi_col_trans(data, cols, id='id', rn='rn'):
    trans_data = data.groupby(id)[rn].max().reset_index(drop=False)
    trans_data = pd.merge(trans_data, data.groupby(id)[[*cols]].sum().reset_index(drop=False), left_on=id, right_on=id)
    return trans_data

In [ ]:
trans_data = bi_col_trans(df, bi_cols)

In [ ]:
trans_data.info()

In [ ]:
def ohe_trans(data, col):
    ohe = OneHotEncoder(sparse=False)
    df = pd.DataFrame(data[col], columns=[col])
    ohe.fit(df)
    ohe_categorical = ohe.transform(df)
    data[ohe.get_feature_names_out()] = ohe_categorical.astype('int8')
    data = data.drop(col, axis=1)
    data = data.groupby('id')[ohe.get_feature_names_out()].sum().reset_index(drop=False)
    return data

In [ ]:
for col in cat_cols:
    tmp_data = ohe_trans(df[['id', col]], col)
    trans_data = pd.merge(trans_data, tmp_data, left_on='id', right_on='id')
    del tmp_data

In [ ]:
trans_data.info()

In [ ]:
trans_data.isna().sum()

In [ ]:
cols = df.columns.to_list()

for col in cols:
    print(f'{col}, {df[col].nunique()}')

id, 250000
rn, 51
pre_since_opened, 20
pre_since_confirmed, 18
pre_pterm, 18
pre_fterm, 17
pre_till_pclose, 17
pre_till_fclose, 16
pre_loans_credit_limit, 20
pre_loans_next_pay_summ, 7
pre_loans_outstanding, 5
pre_loans_total_overdue, 1
pre_loans_max_overdue_sum, 3
pre_loans_credit_cost_rate, 14
pre_loans5, 8
pre_loans530, 15
pre_loans3060, 5
pre_loans6090, 3
pre_loans90, 4
is_zero_loans5, 2
is_zero_loans530, 2
is_zero_loans3060, 2
is_zero_loans6090, 2
is_zero_loans90, 2
pre_util, 20
pre_over2limit, 20
pre_maxover2limit, 20
is_zero_util, 2
is_zero_over2limit, 2
is_zero_maxover2limit, 2
enc_paym_0, 4
enc_paym_1, 4
enc_paym_2, 4
enc_paym_3, 4
enc_paym_4, 4
enc_paym_5, 4
enc_paym_6, 4
enc_paym_7, 4
enc_paym_8, 4
enc_paym_9, 4
enc_paym_10, 4
enc_paym_11, 4
enc_paym_12, 4
enc_paym_13, 4
enc_paym_14, 4
enc_paym_15, 4
enc_paym_16, 4
enc_paym_17, 4
enc_paym_18, 4
enc_paym_19, 4
enc_paym_20, 4
enc_paym_21, 4
enc_paym_22, 4
enc_paym_23, 4
enc_paym_24, 4
enc_loans_account_holder_type, 7
enc_loans

In [ ]:
bi_cols = []
cat_cols = []
for col in cols:
    if df[col].nunique() > 2:
        cat_cols.append(col)
    else:
        bi_cols.append(col)

print(bi_cols)
print(cat_cols)

In [ ]:
trans_data = df[['id', 'rn', *bi_cols]]

In [ ]:
trans_data = pd.merge(trans_data, tmp_data, left_on='id', right_on='id')
trans_data

Так как признаки бинаризованы (по сути, категориальные несравнимые признаки) и имеют малое количество уникальных значений, используем OHE-кодирование.